# Grafazo: Con que alumnos puedo hacer TPs en un futuro?

Este grafo se encarga de analizar "camadas" de gente: grupos de alumnos que cursaron varias materias juntos. 

Una vez que podamos distinguir esas camadas, ya tenemos posibles compañeros de TPs. Después, dentro de mi propia camada, con quien más quiero hacer un TP es con los alumnos que se parezcan a mí académicamente. Entonces tenemos que pasar a tener en cuenta la nota de las materias. La idea final es que yo me haga compañero de alumnos que tiendan a cursar las mismas materias que yo, y que tengamos el mismo nivel académico.

Ojo, este grafo no apunta a responder *en que* materia hacer tps juntos: eso involucraría fijarse qué curso cada alumno y fijarse que personas todavía no cursaron lo mismo. Ya con solo ser de la misma camada sabemos que nos quedan materias en las que nos vamos a cruzar. La idea es un poco mas generalizada a encontrar compañeros de clase, no importa en que materia.

Por ejemplo: como Rosita y yo cursamos las mismas materias por dos años, somos de la misma camada. Y encima, como siempre nos sacamos notas parecidas, debe ser un buen compañero de TP para mí.

## Cómo es el grafo?

El grafo analizado va a ser un multigrafo: entre cada par de alumnos puede haber varias aristas

- Nodos: alumnos
- Aristas: conectar dos alumnos que hayan cursado la misma materia el mismo cuatrimestre
- Peso de las aristas: la relacion entre las notas de esa cursada. Mientras más parecidos somos, más cercano estamos, y por ende menor peso hay en nuestra arista. Lo calculamos como la diferencia entre las notas.
    - Si me saque un 10 y vos un 10, nuestro peso es 0.
    - Si me saque un 4 y vos un 10, nuestro peso es 6.
    - En el caso de que yo estoy en final y vos aprobaste, hardcodeamos el peso a 7

In [ ]:
import pandas as pd

df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

In [ ]:
def corrnotas(row):
    if ((row['src_nota'] == -1 and row['dst_nota'] != -1) or
        (row['dst_nota'] == -1 and row['src_nota'] != -1)):
        return 7
    return abs(row['src_nota'] - row['dst_nota'])

df_nodes_metadata = df[df.columns & ["Padron", "Carrera", "aplazos", "optativas", "checkboxes"]]
df_nodes_metadata = df_nodes_metadata[df_nodes_metadata["aplazos"].notnull() | df_nodes_metadata["optativas"].notnull()]
df_nodes_metadata = df_nodes_metadata.groupby(["Padron", "Carrera"], as_index=False).first()

df_nodes = df[["Padron", "Carrera", "Orientacion", "Final de Carrera"]]
df_nodes = df_nodes.drop_duplicates()
df_nodes = df_nodes.merge(df_nodes_metadata, how="outer")
df_nodes.set_index("Padron", inplace=True)
display(df_nodes.dropna().sample(3))

from itertools import combinations
df_edges = (df[df['materia_cuatrimestre'].notnull()]
     .groupby(['materia_id', 'materia_cuatrimestre'])[['Padron', 'materia_nota']]
     .apply(lambda x : list(combinations(x.values,2)))
     .apply(pd.Series)
     .stack()
     .reset_index(level=0, name='Usuarios')
)

df_edges = df_edges.reset_index()
df_edges[['src', 'dst']] = df_edges['Usuarios'].tolist()
df_edges[['src_padron', 'src_nota']] = df_edges['src'].tolist()
df_edges[['dst_padron', 'dst_nota']] = df_edges['dst'].tolist()

# Nos quedamos solo con las materias aprobadas (nota > 0) o en final (-1)
df_edges = df_edges[(df_edges['src_nota'] != -2) & (df_edges['src_nota'] != 0)]
df_edges = df_edges[(df_edges['dst_nota'] != -2) & (df_edges['dst_nota'] != 0)]

# Calculamos la correlacion entre las notas
df_edges['corrnotas'] = df_edges.apply(corrnotas, axis=1)

df_edges = df_edges[['src_padron', 'dst_padron', 'materia_cuatrimestre', 'materia_id', 'src_nota', 'dst_nota', 'corrnotas']]
display(df_edges.dropna().sample(3))

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import utils

G = nx.from_pandas_edgelist(df_edges, 
                            source='src_padron', 
                            target='dst_padron', 
                            edge_attr=['materia_id','materia_cuatrimestre', 'corrnotas'], 
                            create_using=nx.MultiGraph())

nx.set_node_attributes(G, df_nodes.to_dict('index'))

print(G)
print(f"""
  El diámetro de la red: {nx.diameter(G)}
  El grado promedio de la red: {sum([n[1] for n in G.degree()]) / len(G):.2f}
  Puentes globales: {list(nx.bridges(G))}
""")

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(30,10))
ax = axes.flatten()

nx.draw_networkx(G, pos=nx.circular_layout(G), width=0.005, node_size=50, with_labels=False, ax=ax[0])

utils.plot_diametro(G)

In [ ]:
# ToDo: graficar un sample con esto!

# # Solo como demo para cuando tenemos pocos nodos: ahora veamos el grafo, pero dibujemos todas las aristas
# # Robadisimo de: https://stackoverflow.com/a/60638452
# pos = nx.random_layout(G)
# nx.draw_networkx_nodes(G, pos)
# ax = plt.gca()
# for e in G.edges:
#     ax.annotate("",
#                 xy=pos[e[0]], xycoords='data',
#                 xytext=pos[e[1]], textcoords='data',
#                 arrowprops=dict(arrowstyle="-", color="0.5",
#                                 connectionstyle="arc3,rad=rr".replace('rr',str(0.3*e[2]))),
#                 )
# plt.axis('off')
# plt.show()

# Homofilia

> La homofilia nos explica otra forma en la que los vínculos se forman. Esto puede depender de diferentes características, por ejemplo podemos unir personas por género, edad, nacionalidad, intereses, creencias

Cuando pensamos en personas en la vida cotidiana y cómo estas podrían vincularse, lo más natural es pensar en edad, intereses, ideología, etc. Cuando pensamos en alumnos de una universidad a lo largo de su carrera, no siempre eso es lo que une a la gente. La edad capaz ya no es un factor central, ya que desde la primera materia de cualquier carrera podemos encontrar personas recién salidas del secundario, como gente de mediana y alta edad. Cuando nos encontramos buscando una característica que vincule a los alumnos, lo primero que pensamos es en las **camadas**, aquellas personas con las que tendes a cursar más materias.

Para realizar este análisis vamos a dividir al alumnado en 4 camadas. La camada número 1 estará compuesta por aquellos padrones que comienzan con 100 o inferiores. Las siguientes camadas son desde los padrones 101 a 104; de 105 a 109;y de 110 a 120(siendo esta bastante chica por ahora).

In [ ]:
df_camadas = df_edges.copy()

# ToDo: Preferimos hacerlo con quantiles? Al tratarse de homofilia siento que esta bien por rangos pero como quieran
def definir_camada(f):
    if not f.isnumeric():
        return 0
    f = int(f)
    if f in range(900,999) or f == 100:
        return 1
    if f in range(101, 104):
        return 2
    if f in range(105, 109):
        return 3
    if f in range(110, 120):
        return 4
    return 0

df_camadas['src_camada'] = df_camadas.apply(lambda f: definir_camada(str(f.src_padron)[:3]), axis=1) 
df_camadas['dst_camada'] = df_camadas.apply(lambda f: definir_camada(str(f.dst_padron)[:3]), axis=1)
df_camadas = df_camadas[df_camadas['src_camada'] > 0]
df_camadas = df_camadas[df_camadas['dst_camada'] > 0]

df_camadas.sample(4)

#### Valores esperados

$pCamadaX = # nodos de Camada X / # nodos totales$

- $pCamada1 = 15064 / 53534 = 0,281 $
- $pCamada2 = 19242 / 53534 = 0,359 $
- $pCamada3 = 18690 / 53534 = 0,349 $
- $pCamada4 = 538 / 53534 = 0,01 $

Por lo que la probabilidad de ser de la camada 1 y compartir con alguien de la camada 1 es $pCamada1^2$, mientras que cruzarse entre camadas te da una probabilidad de $pCamada1*pCamada2*2$
|          | Camada 1 | Camada 2 | Camada 3 | Camada 4 |
|----------|----------|----------|----------|----------|
| Camada 1 | 0,078    | 0,1      | 0,098    | 0,003    |
| Camada 2 | 0,1      | 0,129    | 0,125    | 0,0035   |
| Camada 3 | 0,098    | 0,125    | 0,122    | 0,0034   |
| Camada 4 | 0,003    | 0,0035   | 0,0034   | 0.0001   |

#### Valores encontrados

In [ ]:
df_camadas.groupby(['src_camada', 'dst_camada']).count()[["src_padron"]].rename(columns={'src_padron':'Count'})

Los valores encontrados muestran claramente lo que planteamos como teoría. Aquellos alumnes que tienen padrones similares tienden a encontrarse más en las cursadas.

## Comunidades

In [ ]:
from networkx.algorithms import community
import matplotlib.colors as mcolors
import random

random.seed(42)
plt.figure(figsize=(30,10))

# La primera corrida solo calculamos camadas, sin darle peso a las notas. Pasamos `weight=None` a louvain
louvain = community.louvain_communities(G, weight=None)
draw_nodes = {}
colors = random.sample(list(mcolors.CSS4_COLORS), len(louvain))
for louvaincommunity, color in zip(louvain, colors):
    draw_nodes.update({n: color for n in louvaincommunity})
    
plt.title(f"{len(louvain)} Louvain Communities")
nx.draw_networkx(G, 
                 nodelist=draw_nodes.keys(), 
                 node_color=list(draw_nodes.values()), 
                 width=0.005, 
                 pos=nx.kamada_kawai_layout(G),
                 font_size=10)

### Evaluación de comunidades por "generación"

Ahora que ya calculamos nuestras comunidades tenemos que confirmar. Efectivamente se refieren a distintas camadas de alumnos? Alumnos que ingresaron a la facultad al mismo tiempo, por ejemplo?

¿Existe correlación entre la distribución de padrones y la comunidad? De tal forma que las comunidades sean por camadas o "generacionales"

In [ ]:
import seaborn as sns
import numpy as np

louvain_padrones = []
for i, comunidad in enumerate(louvain):
    for padron in comunidad:
        louvain_padrones.append((padron, i))
df_comunidades = pd.DataFrame(louvain_padrones, columns=["padron", "comunidad"])

# len patch for overflow
df_temp = df_comunidades[(df_comunidades['padron'].str.isdigit()) & (df_comunidades["padron"].str.len() >= 5) & (df_comunidades["padron"].str.len() <= 6)].copy()
df_temp["padron"] = df_temp["padron"].astype(int)

# remove outliers by percentiles, stolen from https://stackoverflow.com/a/59366409
Q1 = df_temp["padron"].quantile(0.10)
Q3 = df_temp["padron"].quantile(0.90)
IQR = Q3 - Q1
df_comunidades = df_temp[~((df_temp["padron"] < (Q1 - 1.5 * IQR)) |(df_temp["padron"] > (Q3 + 1.5 * IQR)))]

display(df_comunidades.sample(3))
display(df_comunidades.groupby('comunidad').agg({'padron':[np.mean,np.std,'count']}))

# TODO: play with the "stat" parameter. another interesting one is the "density" parameter
g = sns.displot(
    df_comunidades,
    x="padron",
    col="comunidad",
    element="step",
    stat="count",
    common_norm=False,
)

Se puede observar una mínima correlación considerando el intervalo más frecuente de cada comunidad. Esto coincide con lo que se planteo con el concepto de homofilia.

### Materias y cuatrimestres en común

Se puede aproximar a una mejor noción de _camada_ que simplemente tomando el padrón, tomando el conjunto de materias/cuatrimestre más populares por comunidad.

- todo: explicar esto

In [ ]:
df = df_comunidades.copy()
df['padron'] = df['padron'].astype(str)
df_src = pd.merge(df_edges, df, left_on='src_padron', right_on='padron')
df_dst = pd.merge(df_edges, df, left_on='dst_padron', right_on='padron')
(
    pd
        .concat([df_src, df_dst])
        .groupby(['comunidad', 'materia_cuatrimestre', 'materia_id'])
        .agg(count=('src_padron', 'count'))
        .sort_values('count', ascending=False)
        .reset_index()
        .groupby('comunidad')   # obtener top 3 materias/cuatrimestres por comunidad
        .head(3)
        .sort_values(['comunidad', 'count'])
)

## Alumnos similares dentro de la misma camada

Ahora que ya tenemos cada subgrafo de gente que cursó junta, queremos encontrar alumnos similares en sus notas. O sea, ya se que soy parte de una camada de 100 personas. De esas 100, con quien me conviene hacer un TP?

Entonces vamos a volver a calcular comunidades, pero esta vez teniendo en cuenta el peso de las aristas, que representan la similitud académica.

todo: deberiamos, de la misma forma que lo hacemos con las comunidades, confirmar que estas subcomunidades tambien se clasifican segun algo (por ejemplo, calcular el avg y std de corrnotas de cada subcomunidad y mostrar que son distintos)

In [ ]:
from setup import PADRON

def nestedsearch(el, lst_of_sets):
    return list(filter(lambda lst: el in lst, lst_of_sets))[0]

def armar_grupo(padron):
    camada = nestedsearch(padron, louvain)       
    subnetwork = nx.subgraph(G, camada)
    min_alumnos, max_alumnos = 6, 14
    max_iteraciones = 25
    
    i = 0
    grupo = []
    for i in range(max_iteraciones):
        sublouvains = community.louvain_communities(subnetwork, weight='corrnotas', resolution=1+(i*0.01))
        comunidad = nestedsearch(padron, sublouvains)
        if min_alumnos <= len(comunidad) <= max_alumnos:
            grupo = comunidad
            break
        elif not grupo or (len(comunidad) >= max_alumnos and (len(comunidad) - max_alumnos <= len(grupo) - max_alumnos)):
            grupo = comunidad
        i+=1

    grupo.remove(padron)
    return grupo

grupo = armar_grupo(PADRON)

plt.figure(figsize=(30,10))
plt.title(f"Posibles compañeros de TP de {PADRON}")
nx.draw_networkx(nx.subgraph(G, grupo), 
                 width=0.01, 
                 font_size=10)

display(pd.DataFrame(list(grupo)))